In [60]:
import pandas as pd
import requests
import os

In [61]:
df = pd.read_csv(os.path.join(*'../../datasets/zalando_outfit_detail_pages/woman/20240310-142422.csv'.split('/')))
df

,Unnamed: 0,outfit_detail_url,outfit_type
0,0,https://www.zalando.ch/outfits/5Bay-ErCSwa/,sporty
1,1,https://www.zalando.ch/outfits/p_pBMObnSJC/,sporty
2,2,https://www.zalando.ch/outfits/lqdPJ6bSSvi/,sporty
3,3,https://www.zalando.ch/outfits/KVp2TJLCRKK/,sporty
4,4,https://www.zalando.ch/outfits/W72I8-8XSwK/,sporty
...,...,...,...
6941,6941,https://www.zalando.ch/outfits/wg2YGdLtRmS/,classic
6942,6942,https://www.zalando.ch/outfits/ywabM2KNSPK/,classic
6943,6943,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,classic
6944,6944,https://www.zalando.ch/outfits/IMi9cMhhTSm/,classic


In [62]:
first_item = df.loc[0]
first_item

Unnamed: 0                                                     0
outfit_detail_url    https://www.zalando.ch/outfits/5Bay-ErCSwa/
outfit_type                                               sporty
Name: 0, dtype: object

In [63]:

page_content = requests.get(first_item['outfit_detail_url'])
page_content

<Response [200]>

In [64]:
page_content.text

'<!DOCTYPE html>\n<html class="rendering-engine re-main-2837 release-main-5228" lang="de-CH">\n<head>\n<meta charset="utf-8"/>\n<meta http-equiv="X-UA-Compatible" content="IE=Edge"/>\n<meta http-equiv="Accept-CH" content="DPR, Viewport-Width, Width">\n<meta name="viewport" content="width=device-width, initial-scale=1" />\n<link rel="preconnect" href="https://mosaic02.ztat.net/" crossorigin>\n<link rel="preconnect" href="https://img01.ztat.net/">\n<link rel="preconnect" href="https://mosaic01.ztat.net/">\n<link rel="preconnect" href="https://mosaic02.ztat.net/">\n<link rel="preconnect" href="https://mosaic03.ztat.net/">\n<link rel="stylesheet" href="https://mosaic01.ztat.net/base-assets/8-0-4/assets/style.css" />\n<link rel="stylesheet" href="https://mosaic01.ztat.net/base-assets/8-0-4/assets/z-shop-ui-styles.css" />\n<link rel="stylesheet" href="https://mosaic02.ztat.net/zds/dx-ui/lib/atom-75.3.12-74.css" /><style>\n@font-face {\n  font-family: "76a99f7";\n  font-display: block;\n  src

In [65]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page_content.content, "html.parser")

In [66]:
clothes = soup.select('.z5x6ht._0xLoFW.JT3_zV.mo6ZnF.I7OI1O.C3wGFf')

In [67]:
clothing_item_tag = clothes[0]
clothing_item_tag

<article class="z5x6ht _0xLoFW JT3_zV mo6ZnF I7OI1O C3wGFf" role="link"><div class="xCblER KVKCn3 u-C3dd jDGwVr mo6ZnF KLaowZ rQ5pcU Nx0a9q"></div><div class="L5YdXz _0xLoFW _7ckuOK mROyo1 S3xARh"><a class="_LM JT3_zV CKDt_l _5qdMrS bAjxtD Sw_aP_ _9hq5vx CKDt_l LyRfpJ" data-card-type="media" href="https://www.zalando.ch/puma-ess-trainingsjacke-puma-black-pu141f06g-q11.html" tabindex="0"><figure class="heWLCX"><div class="JT3_zV ZkIJC- _9QaS6n IN7Kbz K82if3"><div class="KVKCn3 u-C3dd jDGwVr mo6ZnF KLaowZ"><img alt="PADDED VEST - Weste - black" class="sDq_FX lystZ1 FxZV-M _2Pvyxl JT3_zV EKabf7 mo6ZnF _1RurXL mo6ZnF _7ZONEy" draggable="false" src="https://img01.ztat.net/article/spp-media-p1/f06f0508a2bd40a6a314e08eb122146f/55ae123cf67f4d8d9b82b2d6fc144965.jpg?imwidth=200&amp;filter=packshot"/></div></div></figure></a><div class="_5qdMrS KVKCn3 _2dqvZS hN9H_L df4QKn _LM"><button aria-label="Artikel auf den Wunschzettel setzen oder davon entfernen" aria-pressed="false" class="BXKCcR r9BRio 

In [68]:
print(clothing_item_tag.select_one('a').get('href'))

https://www.zalando.ch/puma-ess-trainingsjacke-puma-black-pu141f06g-q11.html


In [69]:

from bs4 import Tag


class ClothingItem:

    def __init__(self, beautiful_soup_item: Tag):
        self._name_tag_selector = ".sDq_FX.lystZ1.FxZV-M.HlZ_Tf.ZkIJC-.r9BRio.qXofat.EKabf7"
        self._brand_tag_selector = ".FtrEr_.lystZ1.FxZV-M.HlZ_Tf.ZkIJC-.r9BRio.qXofat.EKabf7"

        self.name = beautiful_soup_item.select_one(self._name_tag_selector).get_text()
        self.preview_image_src = beautiful_soup_item.find('img').get('src')
        self.brand = beautiful_soup_item.select_one(self._brand_tag_selector).get_text()
        self.clothing_item_detail_url = beautiful_soup_item.select_one('a').get('href')

    def __str__(self):
        return f'ClothingItem(name={self.name}, brand={self.brand}, url={self.preview_image_src}, detail_url={self.clothing_item_detail_url}'

    def __repr__(self):
        return self.__str__()


def get_values_from_clothing_item_link(html):
    print(ClothingItem(html))


class ClothingItemParser:

    @property
    def clothes(self):
        return self._clothes

    def __init__(self, html):
        self._clothes = []
        self._clothing_item_selector = ".z5x6ht._0xLoFW.JT3_zV.mo6ZnF.I7OI1O.C3wGFf"
        soup = BeautifulSoup(html, "html.parser")

        clothes = soup.select(self._clothing_item_selector)

        for clothing_item_tag in clothes:
            self._clothes.append(ClothingItem(clothing_item_tag))






In [70]:
clothes = ClothingItemParser(page_content.content)._clothes

In [71]:
clothes[0]

ClothingItem(name=PADDED VEST - Weste - black, brand=Puma, url=https://img01.ztat.net/article/spp-media-p1/f06f0508a2bd40a6a314e08eb122146f/55ae123cf67f4d8d9b82b2d6fc144965.jpg?imwidth=200&filter=packshot, detail_url=https://www.zalando.ch/puma-ess-trainingsjacke-puma-black-pu141f06g-q11.html

In [72]:
def get_clothing_items_for_oufit_url(url):
    print(f'fetch url: {url}')
    return ClothingItemParser(requests.get(url).content).clothes


In [73]:
clothing_items = get_clothing_items_for_oufit_url(first_item['outfit_detail_url'])
clothing_items

fetch url: https://www.zalando.ch/outfits/5Bay-ErCSwa/


[ClothingItem(name=PADDED VEST - Weste - black, brand=Puma, url=https://img01.ztat.net/article/spp-media-p1/f06f0508a2bd40a6a314e08eb122146f/55ae123cf67f4d8d9b82b2d6fc144965.jpg?imwidth=200&filter=packshot, detail_url=https://www.zalando.ch/puma-ess-trainingsjacke-puma-black-pu141f06g-q11.html,
 ClothingItem(name=SLFMY PERFECT TEE BOX CUT - T-Shirt basic - bright white, brand=Selected Femme, url=https://img01.ztat.net/article/spp-media-p1/3b90733d789c33dbb1c38f172a15e198/dc552ad9ec694d3dab08571b7dbe8309.jpg?imwidth=200&filter=packshot, detail_url=https://www.zalando.ch/selected-femme-sfmy-t-shirt-basic-bright-white-se521d07g-a11.html,
 ClothingItem(name=MOM A LINE SHORT - Jeans Shorts - flash black, brand=Levi's®, url=https://img01.ztat.net/article/spp-media-p1/eb53cdf956e73aadbdd92c945814690e/1113685ce19b425e9d048f01f0ddec80.jpg?imwidth=200&filter=packshot, detail_url=https://www.zalando.ch/levisr-mom-a-line-short-jeans-shorts-flash-black-le221s01a-q11.html,
 ClothingItem(name=530 UNI

## Start Scraping

In [74]:
number_of_clothes = df.shape[0]
number_of_clothes

6946

In [75]:
from typing import List
from time import sleep
from tqdm import tqdm
import random

number_of_clothes = df.shape[0]

clothing_items_for_df = []

print("Start crawling outfits")

for i in tqdm(range(number_of_clothes)):
    #for i in tqdm(range(3)):
    outfit = df.loc[i]
    clothing_items = []
    clothing_item_url = outfit['outfit_detail_url']
    clothing_items = get_clothing_items_for_oufit_url(clothing_item_url)

    for clothing_item in clothing_items:
        clothing_item: ClothingItem = clothing_item
        print(clothing_item)

        clothing_items_for_df.append(
            # ['outfit_url',   'clothingItemName', 'clothingItemBrand', 'clothingItemDetailUrl', 'clothingItemPreviewImageUrl']
            (clothing_item_url, clothing_item.name, clothing_item.brand, clothing_item.clothing_item_detail_url,
             clothing_item.preview_image_src)
        )

    sleep(random.uniform(3, 8))

df_clothing_items = pd.DataFrame(clothing_items_for_df, columns=['outfit_url', 'clothingItemName', 'clothingItemBrand',
                                                                 'clothingItemDetailUrl',
                                                                 'clothingItemPreviewImageUrl'])

df_clothing_items.to_csv('woman_outfit_clothes.csv')

print("Finished crawling, wrote csv file")

Start crawling outfits


  0%|          | 0/3 [00:00<?, ?it/s]

fetch url: https://www.zalando.ch/outfits/5Bay-ErCSwa/
ClothingItem(name=PADDED VEST - Weste - black, brand=Puma, url=https://img01.ztat.net/article/spp-media-p1/f06f0508a2bd40a6a314e08eb122146f/55ae123cf67f4d8d9b82b2d6fc144965.jpg?imwidth=200&filter=packshot, detail_url=https://www.zalando.ch/puma-ess-trainingsjacke-puma-black-pu141f06g-q11.html
ClothingItem(name=SLFMY PERFECT TEE BOX CUT - T-Shirt basic - bright white, brand=Selected Femme, url=https://img01.ztat.net/article/spp-media-p1/3b90733d789c33dbb1c38f172a15e198/dc552ad9ec694d3dab08571b7dbe8309.jpg?imwidth=200&filter=packshot, detail_url=https://www.zalando.ch/selected-femme-sfmy-t-shirt-basic-bright-white-se521d07g-a11.html
ClothingItem(name=MOM A LINE SHORT - Jeans Shorts - flash black, brand=Levi's®, url=https://img01.ztat.net/article/spp-media-p1/eb53cdf956e73aadbdd92c945814690e/1113685ce19b425e9d048f01f0ddec80.jpg?imwidth=200&filter=packshot, detail_url=https://www.zalando.ch/levisr-mom-a-line-short-jeans-shorts-flash-bl

 33%|███▎      | 1/3 [00:06<00:12,  6.44s/it]

fetch url: https://www.zalando.ch/outfits/p_pBMObnSJC/
ClothingItem(name=ONPDEEP LIFE OPEN BACK - Langarmshirt - windsor wine, brand=ONLY Play, url=https://img01.ztat.net/article/spp-media-p1/37fc19d22e4749c6a433604e8c486f64/4bec296c58854a7bb79d399d29285d4d.jpg?imwidth=200&filter=packshot, detail_url=https://www.zalando.ch/only-play-onpdeep-life-open-back-langarmshirt-windsor-wine-nl241d0uj-g11.html
ClothingItem(name=ONPTAMI LASER CUT SPORTS BRA - Sport-BH mit mittlerer Stützkraft - orion blue, brand=ONLY Play, url=https://img01.ztat.net/article/spp-media-p1/fa3fec4b6d864520b11f7dae00263359/a79f4f27281e4d27989292cb03eb9517.jpg?imwidth=200&filter=packshot, detail_url=https://www.zalando.ch/only-play-sport-bh-mit-mittlerer-stuetzkraft-orion-blue-nl241i08j-k11.html
ClothingItem(name=SEAMLESS  - Tights - light brown, brand=Even&Odd active, url=https://img01.ztat.net/article/spp-media-p1/6e8e08339cef4ecc8bacccf4dbd6c70c/c85b244c03e94067afcc0b32579fc809.jpg?imwidth=200&filter=packshot, detai

 67%|██████▋   | 2/3 [00:15<00:07,  7.83s/it]

fetch url: https://www.zalando.ch/outfits/lqdPJ6bSSvi/
ClothingItem(name=HYPERGLAM TRAINING QUARTER ZIP TOP - Langarmshirt - semi green spark/white, brand=adidas Performance, url=https://img01.ztat.net/article/spp-media-p1/4c761c238e444dd6ba68479bdc9dfb2b/85e559bc0ffd4bbca3d4218d24d40cf3.jpg?imwidth=200&filter=packshot, detail_url=https://www.zalando.ch/adidas-performance-hyperglam-quarter-zip-langarmshirt-semi-green-sparkwhite-ad541d2oo-m11.html
ClothingItem(name=UNIVERSA - Tights - baroque brown/black, brand=Nike Performance, url=https://img01.ztat.net/article/spp-media-p1/9d260f2eb23344569f96cb963c3e7a21/48914616634a4ef49a74c71c6e5f96be.jpg?imwidth=200&filter=packshot, detail_url=https://www.zalando.ch/nike-performance-universa-tights-baroque-brownblack-n1241e1l5-o11.html
ClothingItem(name=ULTRABOOST LIGHT - Laufschuh Natural running - crystal jade/zero metallic/green spark, brand=adidas Performance, url=https://img01.ztat.net/article/spp-media-p1/f65de37733434ef48a651305cc765040/0f

100%|██████████| 3/3 [00:20<00:00,  6.90s/it]

Finished crawling, wrote csv file
